# 随机指标KDJ
随机指标KDJ，又称随机指数，是一种用来分析市场中超买超卖现象。

KDJ指标最基础交易思想是建立在 威廉指标之上，对资产进行分析时，除考虑资产每日收盘价格外，还要综合分析日间交易时间价格的变化情况。

随机指标KDJ，使用特定时间跨度中的最后收盘价与该时间段跨度内的最高价、最低价来推测市场的超买和超卖情况。KDJ指标结合统计学原理和移动平均的思想绘制K、D、J线，综合运用相对强度指标、动量指标和移动平均指标的特点更加直观、清晰绘制出股票走势。

KDJ指标本质上是一个随机波动的观念，其对于中短期行情的预测较为准确。

## 随机指标计算公式

KDJ指标由K、D、J三条线组成。首先根据特定周期证券的最高价、最低价、最后一个计算时点的收盘价计算最后一个时点的RSV（未成熟随机值），再通过移动平均法来计算K、D、J值。

### RSV
若用$Close_t$ 表示t时期的收盘价，$High(t-n+1,t)$  表示 t-n+1日到第t日n个交易日的最高价，$Low(t-n+1,t)$ 表示t-n+1日到第t日n个交易日最低价，那么，$RSV_t$ 的计算公式：
$$
\begin{aligned}
& RSV_{t} = \frac{Close_{t} - Low_{(t-n+1,t)}}{High_{(t-n+1,t)} - Low_{(t-n+1,t)}}
\end{aligned}
$$
其中，n为时间跨度，通常取值为9日或9周。

RSV取值范围0～100之间。
- RSV 取值越大，说明收盘价在价格区间中的相对位置较高，市场出现超买现象，释放卖出信号；
- RSV 取值越小，说明收盘价的相对位置较低，市场出现超卖现象，释放出买入信号。

### K 、D、J

K值，由前一日的K值和当前RSV值在通过权重调整计算得到。
$$
\begin{aligned}
K_{t} = \frac{2}{3} * K_{t-1} + \frac{1}{3}  * RSV_{t}
\end{aligned}
$$
D值，由前一日的D值和当期K值结果一定权重调整计算得到。
$$
\begin{aligned}
D_t = \frac{2}{3} * D_{t-1} + \frac{1}{3} * K_t
\end{aligned}
$$
J值，是KD指标的辅助指标，反映了K和D指标乖离程度。
$$
\begin{aligned}
J_t = 3 * K_t - 2 * D_t
\end{aligned}
$$

## 随机指标应用说明
1. K、D 的取值范围：0～100；J的取值，可以大于100，也可以低于0
2. K、D、J值说明：
		K或D > 80 ，当前行情为超买情况，释放卖出信号；
		K或D < 20 ，当前行情为超卖情况，释放买入信号；
		J > 100 ，当前行情进入超买情况，释放卖出信号；
		J < 0 ， 当前行情进入超卖情况，释放买入信号；
3. K、D交叉情况：
		“黄金交叉”：K线由下向上穿过D线，股票上涨动量较大，释放买入信号；
		“死亡交叉”：K线由上向下穿过D线，股票下跌动量较大，释放卖出信号。

# KDJ交易策略
## 策略规则
根据随机指标运用：
1. K、D的取值范围：0～100；J的取值，可以大于100，也可以低于0
2. K、D、J值说明：
		K或D > 80 ，当前行情为超买情况，释放卖出信号；
		K或D < 20 ，当前行情为超卖情况，释放买入信号；
		J > 100 ，当前行情进入超买情况，释放卖出信号；
		J < 0 ， 当前行情进入超卖情况，释放买入信号；
3. K、D交叉情况：
		“黄金交叉”：K线由下向上穿过D线，股票上涨动量较大，释放买入信号；
		“死亡交叉”：K线由上向下穿过D线，股票下跌动量较大，释放卖出信号。

在这里，601318 中国平安 为例，计算其2022-01-01 至 2023-01-01期间每日行情的MA5（短期均线）、MA20（长期均线）。

In [1]:
import sys
sys.path.append(r'/Users/paul/DSWorkspace/001量化策略/002-Project项目/lleasy')

# 1. 数据准备
from lleasy.database import SqliteDatabase as sqlitedb
from lleasy.object import TradeData, BarData
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

## 数据准备

In [2]:
# 获取数据
# 标的：601318 中国平安
# 时间：2022-01-01 至 2023-01-01
# 频率：日行情
#获取sqlite链接
db = sqlitedb()
datas= db.get_symbol_bars_bySEDate('601318','2022-01-01','2023-01-01')
datas=datas.set_index(['datetime'])

2023-08-05 15:33:08,453 DEBUG sqlalchemy.pool.impl.QueuePool Created new connection <sqlite3.Connection object at 0x11e1f5d40>
2023-08-05 15:33:08,454 DEBUG sqlalchemy.pool.impl.QueuePool Connection <sqlite3.Connection object at 0x11e1f5d40> checked out from pool
2023-08-05 15:33:08,455 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-05 15:33:08,458 INFO sqlalchemy.engine.Engine SELECT llbardata.id AS llbardata_id, llbardata.symbol AS llbardata_symbol, llbardata.interval AS llbardata_interval, llbardata.open AS llbardata_open, llbardata.close AS llbardata_close, llbardata.high AS llbardata_high, llbardata.low AS llbardata_low, llbardata.vol AS llbardata_vol, llbardata.turnover AS llbardata_turnover, llbardata.asset AS llbardata_asset, llbardata.adjust AS llbardata_adjust, llbardata.datetime AS llbardata_datetime 
FROM llbardata 
WHERE llbardata.symbol = ? AND llbardata.datetime >= ? AND llbardata.datetime <= ?
2023-08-05 15:33:08,459 INFO sqlalchemy.engine.Engine [generated in 0

## 编制策略

In [34]:
kdj_bars = datas
kdj_bars['low'] = kdj_bars['low'].astype(float)
kdj_bars['high'] = kdj_bars['high'].astype(float)
kdj_bars['close'] = kdj_bars['close'].astype(float)

#计算时间跨度为9日RSV、K、D、J
kdj_bars[['high_max','low_min','RSV','K','D','J']] = 0
period_high_max = 0
period_low_min = 0

for j in range(8,len(kdj_bars)):
    period_high_max = kdj_bars['high'].iloc[j-8:j+1].max()
    period_low_min = kdj_bars['low'].iloc[j-8:j+1].min()
    
    kdj_bars['high_max'].iloc[j] = period_high_max
    kdj_bars['low_min'].iloc[j] = period_low_min

kdj_bars['RSV'] = 100 * (kdj_bars['close'] - kdj_bars['low_min']) / (kdj_bars['high_max'] - kdj_bars['low_min'])
kdj_bars['K'].iloc[0:8] = 50
kdj_bars['D'].iloc[0:8] = 50

kdj_bars['K'] = 2/3 * kdj_bars['K'].shift(1) + 1/3 * kdj_bars['RSV']
kdj_bars['D'] = 2/3 * kdj_bars['D'].shift(1) + 1/3 * kdj_bars['K']
kdj_bars['J']= 3 * kdj_bars['K'] - 2 * kdj_bars['D']

bars = kdj_bars[['open','close','high','low','K','D','J']]
bars


/var/folders/d9/rbb57jqn3g133_ynk3vgz02c0000gn/T/ipykernel_40078/1960760726.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/d9/rbb57jqn3g133_ynk3vgz02c0000gn/T/ipykernel_40078/1960760726.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/d9/rbb57jqn3g133_ynk3vgz02c0000gn/T/ipykernel_40078/1960760726.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/d9/rbb57jqn3g133_ynk3vgz02c0000gn/T/ipy

,symbol,open,close,high,low,RSV,high_max,low_min,K,D,J
datetime,,,,,,,,,,,
2022-01-04,601318,50.400,51.00,51.06,50.10,inf,0.00,0.00,50,50,0
2022-01-05,601318,51.110,52.07,52.40,50.97,inf,0.00,0.00,50,50,0
2022-01-06,601318,51.990,51.30,52.02,51.22,inf,0.00,0.00,50,50,0
2022-01-07,601318,51.490,52.94,53.33,51.45,inf,0.00,0.00,50,50,0
2022-01-10,601318,52.990,53.08,53.56,52.50,inf,0.00,0.00,50,50,0
2022-01-11,601318,53.170,52.86,53.74,52.54,inf,0.00,0.00,50,50,0
2022-01-12,601318,52.600,52.50,52.81,51.86,inf,0.00,0.00,50,50,0
2022-01-13,601318,52.550,52.30,53.68,52.30,inf,0.00,0.00,50,50,0
2022-01-14,601318,52.350,51.10,52.50,51.06,27.472527,53.74,50.10,0,0,0


In [35]:
# 图形化显示
import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = make_subplots(
    rows=1,cols=1,
    shared_xaxes=True,
)

trace1 = go.Scatter(
    x=bars.index,
    y=bars['K'],
    mode='lines',
    name='K',
    marker={
        'size':8,
#        'color':'rgba(102,198,147,0.7)',
    }
)
trace2 = go.Scatter(
    x=bars.index,
    y=bars['D'],
    mode='lines',
    name='D',
    marker={
        'size':8,
#        'color':'rgba(252,108,117,1)'
    }
)

trace3 = go.Scatter(
    x=bars.index,
    y=bars['J'],
    mode='lines',
    name='J',
    marker={
        'size':8,
#        'color':'rgba(252,108,117,1)'
    }
)

trace4 = go.Scatter(
    x=bars.index,
    y=bars['close'],
    mode='lines',
    name='close',
    marker={
        'size':8,
#        'color':'rgba(252,108,117,1)'
    }
)

fig.add_traces(
    trace1,
    rows=1,cols=1
)
fig.add_traces(
    trace2,
    rows=1,cols=1
)
fig.add_traces(
    trace3,
    rows=1,cols=1
)
fig.add_traces(
    trace4,
    rows=1,cols=1
)
#fig = go.Figure(data=[trace1,trace2,trace3,trace4])

fig.update_xaxes(
        # 在x轴上，去除周六、周日
    rangebreaks=[
        dict(bounds=['sat','mon']),
    ]
)

fig.update_layout(
    hovermode="x"
)

fig.show()


## 执行策略
## 策略回测

# 总结